# Install dependencies


In [ ]:
! git clone https://github.com/GryffindorLi/CS4240_Deep_Learning_Reproduce.git
! pip install -r CS4240_Deep_Learning_Reproduce/requirements.txt

# Check the tokenizer


In [ ]:
from transformers import AlbertTokenizer, AlbertModel
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
# model = AlbertModel.from_pretrained("albert-base-v2")
text = "fairness harm"
max_length = -3
ll = {"a":["equity", "amoral", "purity", "degradation", "loyalty", 
              "care", "cheating", "betrayal", "sabotage", "authority", "harm"]}
for i in ll:
  for j in ll[i]:
    encoded_input = tokenizer.convert_ids_to_tokens(tokenizer.encode(j))
    
    # if max_length < len(encoded_input)-2 :
    print(encoded_input)
    # max_length = len(encoded_input)-2
max_length

# Run the code

In [ ]:
! rm -rf CS4240_Deep_Learning_Reproduce/output
!python CS4240_Deep_Learning_Reproduce/cli.py \
  --method pet --pattern_ids 0 --data_dir CS4240_Deep_Learning_Reproduce/data \
  --model_type albert --model_name_or_path albert-base-v2 --task_name mftc --output_dir CS4240_Deep_Learning_Reproduce/output \
  --do_train --do_eval --pet_per_gpu_eval_batch_size 1  --pet_num_train_epochs 3 --pet_num_train_epochs 3 \
  --train_examples 100 --test_examples 50 --unlabeled_examples 150

In [ ]:
!python cli.py --method pet --pattern_ids 1 --data_dir data_sample --model_type albert --model_name_or_path albert-base-v2 --task_name mftc --output_dir output --do_train --do_eval --pet_per_gpu_eval_batch_size 1 --train_examples 100 --test_examples 50 --unlabeled_examples 150

#Run bert tramsformer

In [ ]:
!pip install SentencePiece
!pip install simpletransformers
import pandas as pd
import sklearn.metrics as skm
from google.colab import files
import os
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
df = pd.read_csv("data/labeled.csv")
cols = df.columns[2:]
from itertools import combinations
combs = combinations(cols, 2)
labels = []
for comb in combs:
  string = ""
  if len(comb) > 1:
      string = comb[0] + " " + comb[1]
  else:
      string = comb[0]
  labels.append(string)

for i in cols: 
  labels.append(i)

print(labels)

def data_loader_dl(datapath):
  df = pd.read_csv(datapath)
  arr = []
  cols = df.columns
  # cols = df.iloc[:,2:]
  for index,row in df.iterrows():
    label = ""
    count = 0
    for index in range(len(row)):
      if row[index] == 1:
        label +=  cols[index] + " "
        count += 1
      if count == 2:
        break
    if label != "" :
      # print(label)
      arr.append(label[:-1])
  df["labels"] = arr
  return df[['text','labels']]


train_set = data_loader_dl("data/labeled.csv")
test_set = data_loader_dl("data/test.csv")

In [ ]:
# delete outputs/ directory
! rm -rf outputs\

# specify hyperparameters and model arguments
model_args = ClassificationArgs(output_dir= "outputs/untrained_model")
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.manual_seed = 42
model_args.use_multiprocessing = True
model_args.train_batch_size = 32
model_args.eval_batch_size = 32
model_args.labels_list = labels

# init bert
model = ClassificationModel('bert', 'bert-base-cased', num_labels=len(labels), args=model_args, use_cuda=True)
# Evaluate the untrained model on test set
result, _ , wrong_predictions = model.eval_model(test_set)

for i in [3]:
  model_args.num_train_epochs = i
  model_args.output_dir = "outputs/epochs_"+str(i)

  # Create a TransformerModel
  model = ClassificationModel('bert', 'bert-base-cased', num_labels=len(labels), args=model_args, use_cuda=True)

  # Train the model
  model.train_model(train_set, show_running_loss=True)

  # Evaluate the model on test set
  result, _ , wrong_predictions = model.eval_model(test_set)
  source = '/content/outputs/epochs_'+str(i)+'/eval_results.txt'
  dst = '/content/outputs/epochs_'+str(i)+'/eval_results_epochs_'+str(i)+'.txt'
  os.rename(source,dst)
  files.download(dst)